# Chunking Examples

## Setup

In [1]:
%pip install -U -q "google-genai>=1.0.0"  # Install the Python SDK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 10.2 MB/s eta 0:00:00


To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](../quickstarts/Authentication.ipynb) quickstart for an example.

In [2]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

MODEL_ID = "gemini-2.5-flash"

In [3]:
from google.genai import types

import requests
import json
import math

questions = requests.get("https://raw.githubusercontent.com/phil-daniel/gemini-batcher/refs/heads/main/examples/demo_files/questions.txt").text.split('\n')
content = requests.get("https://raw.githubusercontent.com/phil-daniel/gemini-batcher/refs/heads/main/examples/demo_files/content.txt").text

# Chunking example 1 - fixed chunking
In fixed chunking, the content is split into non-overlapping chunks each containing a set number of characters, in this case 10,000. An example of this can be seen below.

<img src="https://raw.githubusercontent.com/phil-daniel/gemini-batcher/refs/heads/main/docs/concepts/images/fixed_chunking.svg" alt="A visual example of fixed chunking." width="600">

In [4]:
chunk_char_size = 10000
chunked_content = []
chunk_count = math.ceil(len(content) / chunk_char_size)

for i in range(chunk_count):
    chunk_start_pos = i * chunk_char_size
    chunk_end_pos = min(chunk_start_pos + chunk_char_size, len(content))
    chunked_content.append(content[chunk_start_pos : chunk_end_pos])

print(f'Number of chunks: {len(chunked_content)}')

Number of chunks: 6


## Chunking example 2 - sliding window chunking
One disadvantage of fixed chunking is that since it breaks context at arbitrary positions, important information may get split between chunks, meaning that neither chunk contains enough information to fully answer a question.

A simple solution to this is to follow a sliding window approach, where an overlap (called the window) between adjacent chunks is introduced. This increases the likelihood that a complete answer can be found within a single chunk, however can also increase the total number of chunks.

An example of this can be seen below.

<img src="https://raw.githubusercontent.com/phil-daniel/gemini-batcher/refs/heads/main/docs/concepts/images/sliding_window.svg" alt="A visual example of sliding window chunking." width="600">

In [5]:
chunk_char_size = 10000
window_char_size = 2500

chunked_content = []
chunk_count = math.ceil(len(content) / (chunk_char_size - window_char_size))

for i in range(chunk_count):
    chunk_start_pos = i * (chunk_char_size - window_char_size)
    chunk_end_pos = min(chunk_start_pos + chunk_char_size, len(content))
    chunked_content.append(content[chunk_start_pos : chunk_end_pos])

print(f'Number of chunks: {len(chunked_content)}')

Number of chunks: 8
